In [ ]:
import cv2 #import the library 
import numpy as np
import matplotlib.pyplot as plt
import os


import tensorflow as tf
from statistics import *
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import img_to_array


DATADIR = 'C://IMG//'
CATEGORIES = ["0","1","2","3","4","5","6","7","8","9"]

In [ ]:
training_data = []

def data_augmentation():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  # get the classification

        for img_filename in (os.listdir(path)):  # iterate over each image per digits
            if '.png' not in img_filename:
                continue
            try:
                
                img_array = cv2.imread(os.path.join(path,img_filename),cv2.IMREAD_GRAYSCALE)
                #plt.imshow(img_array, cmap='gray')
                #plt.show()
                
                img_canny = cv2.Canny(img_array, 50, 50)
                
                contours, hierachy = cv2.findContours(img_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    
                    if area > 5:
                        
                        peri = cv2.arcLength(cnt, True)
                        approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
                        
                        x,y,w,h = cv2.boundingRect(approx)
                        
                        image_rect = img_array[y:y+h, x:x+w]
                        image_rect = cv2.resize(image_rect, (100, 100))
                        #plt.imshow(image_rect, cmap='gray')
                        #plt.show()
                        
                        kernel = np.ones((5,5), np.uint8)
                        
                        for blur_value in range(-30, 30):
                            
                            img = cv2.GaussianBlur(image_rect, (7,7), blur_value)
                            training_data.append([img, class_num])
                            
                            
                            
                            img_erosion = cv2.erode(img, kernel, iterations = 1)
                            img_erosion2 = cv2.erode(img, kernel, iterations = 2)
                            training_data.append([img_erosion, class_num])
                            training_data.append([img_erosion2, class_num])
                            
                            
                            
                            
                            img_dilation = cv2.dilate(img, kernel, iterations = 1)
                            img_dilation2 = cv2.dilate(img, kernel, iterations = 2)
                            training_data.append([img_dilation, class_num])
                            training_data.append([img_dilation2, class_num])
                            
                            #plt.imshow(img_dilation2, cmap='gray')
                            #plt.show()
                            
                        
                        
                        
                
                    
                
            except Exception as e:  # in the interest in keeping the output clean...
                raise(e)
            

data_augmentation()

print(len(training_data))



In [ ]:
import random
random.seed(2700)
random.shuffle(training_data)


for features,label in training_data[:1]:
    print(features) 

In [ ]:
IMG_SIZE = 100

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = X.astype('float32')
X = X / 255.0

y = np.array(y)

print(len(X))
print(len(y))

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, epochs = 3)

In [ ]:
model.save('sudoku_model_ytb.h5')